# Evaluating Betting Odds Using Brier Scores

In the previous session we looked at the accuracy of the betting odds when looking at performance of teams across an entire season. Now we look at the accuracy of the betting odds at the individual game level. There are various ways we might do this, but one of the most useful is the Brier Score, which we introduce below.

In [1]:
# install the packages we need

import pandas as pd
import numpy as np

In [2]:
# Let's start by looking at the NBA 2018/19 season again

NBA19 = pd.read_excel("../../Data/Week 2/NBA2019odds.xlsx")
NBA19

,team,opponent,day,month,year,winodds,loseodds,teampts,oppopts,overtime,home,Game(home-away),HAscore,win
0,Atlanta Hawks,New York Knicks,18,10,2018,2.39,1.60,107,126,0,0,New York Knicks - Atlanta Hawks,126:107,0
1,Atlanta Hawks,Memphis Grizzlies,20,10,2018,3.34,1.34,117,131,0,0,Memphis Grizzlies - Atlanta Hawks,131:117,0
2,Atlanta Hawks,Cleveland Cavaliers,22,10,2018,3.97,1.26,133,111,0,0,Cleveland Cavaliers - Atlanta Hawks,111:133,1
3,Atlanta Hawks,Dallas Mavericks,25,10,2018,2.07,1.79,111,104,0,1,Atlanta Hawks - Dallas Mavericks,111:104,1
4,Atlanta Hawks,Chicago Bulls,28,10,2018,1.43,2.91,85,97,0,1,Atlanta Hawks - Chicago Bulls,85:97,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2455,Washington Wizards,Denver Nuggets,1,4,2019,5.68,1.15,95,90,0,0,Denver Nuggets - Washington Wizards,90:95,1
2456,Washington Wizards,Chicago Bulls,4,4,2019,1.19,5.00,114,115,0,1,Washington Wizards - Chicago Bulls,114:115,0
2457,Washington Wizards,San Antonio Spurs,6,4,2019,3.22,1.37,112,129,0,1,Washington Wizards - San Antonio Spurs,112:129,0
2458,Washington Wizards,New York Knicks,8,4,2019,1.58,2.44,110,113,0,0,New York Knicks - Washington Wizards,113:110,0


In [3]:
# Calculate the implied win probability from the decimal odds

NBA19['winprob']= 1/(NBA19['winodds'])/(1/(NBA19['winodds'])+ 1/(NBA19['loseodds']))
NBA19

,team,opponent,day,month,year,winodds,loseodds,teampts,oppopts,overtime,home,Game(home-away),HAscore,win,winprob
0,Atlanta Hawks,New York Knicks,18,10,2018,2.39,1.60,107,126,0,0,New York Knicks - Atlanta Hawks,126:107,0,0.401003
1,Atlanta Hawks,Memphis Grizzlies,20,10,2018,3.34,1.34,117,131,0,0,Memphis Grizzlies - Atlanta Hawks,131:117,0,0.286325
2,Atlanta Hawks,Cleveland Cavaliers,22,10,2018,3.97,1.26,133,111,0,0,Cleveland Cavaliers - Atlanta Hawks,111:133,1,0.240918
3,Atlanta Hawks,Dallas Mavericks,25,10,2018,2.07,1.79,111,104,0,1,Atlanta Hawks - Dallas Mavericks,111:104,1,0.463731
4,Atlanta Hawks,Chicago Bulls,28,10,2018,1.43,2.91,85,97,0,1,Atlanta Hawks - Chicago Bulls,85:97,0,0.670507
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2455,Washington Wizards,Denver Nuggets,1,4,2019,5.68,1.15,95,90,0,0,Denver Nuggets - Washington Wizards,90:95,1,0.168375
2456,Washington Wizards,Chicago Bulls,4,4,2019,1.19,5.00,114,115,0,1,Washington Wizards - Chicago Bulls,114:115,0,0.807754
2457,Washington Wizards,San Antonio Spurs,6,4,2019,3.22,1.37,112,129,0,1,Washington Wizards - San Antonio Spurs,112:129,0,0.298475
2458,Washington Wizards,New York Knicks,8,4,2019,1.58,2.44,110,113,0,0,New York Knicks - Washington Wizards,113:110,0,0.606965


A simple way to evaluate the accuracy of the betting odds is to set some criterion for success and see how often the betting odds satisfy it. A natural criterion when there are two outcomes is to define the betting odds as "correct" if the odds implied the actual outcome and had a probability greater than 50%. 

In [4]:
# Create a dummy variable equal to one when the odds probability is above 50% and zero otherwise.

NBA19['winpred'] = np.where(NBA19['winprob'] >= .5,1,0)

In [5]:
# Identify cases of success based on our criterion, and calculate the average success rate.

NBA19['oddscorrect']= np.where(NBA19['winpred'] == NBA19['win'] ,1,0)
NBA19['oddscorrect'].mean()

0.6731707317073171

On this basis the betting odds were "correct" two thirds of the time. This test implies that if the betting odds implied a 49.999% chance that the team would win, but the team lost, then the odds were "incorrect", whereas it might be more reasonable to think that the odds were indicating that the outcome was just highly uncertain. It's therefore interesting to consider cases where the odds are more decisive. For example, suppose we only consider cases where the outcome (win or lose) is predicted to have a probability of at least 60%? We now define the subset of these cases and derive the success rate.

In [6]:
# Cases where a win or a loss is predicted to have a probability in excess of 60%.

NBA19['highprob'] = np.where(NBA19['winprob'] >= .6,1,np.where(NBA19['winprob'] <= .4,1,0))
NBA19rest = NBA19[NBA19['highprob'] == 1].copy()
NBA19rest

,team,opponent,day,month,year,winodds,loseodds,teampts,oppopts,overtime,home,Game(home-away),HAscore,win,winprob,winpred,oddscorrect,highprob
1,Atlanta Hawks,Memphis Grizzlies,20,10,2018,3.34,1.34,117,131,0,0,Memphis Grizzlies - Atlanta Hawks,131:117,0,0.286325,0,1,1
2,Atlanta Hawks,Cleveland Cavaliers,22,10,2018,3.97,1.26,133,111,0,0,Cleveland Cavaliers - Atlanta Hawks,111:133,1,0.240918,0,0,1
4,Atlanta Hawks,Chicago Bulls,28,10,2018,1.43,2.91,85,97,0,1,Atlanta Hawks - Chicago Bulls,85:97,0,0.670507,1,0,1
5,Atlanta Hawks,Philadelphia 76ers,30,10,2018,6.71,1.12,92,113,0,0,Philadelphia 76ers - Atlanta Hawks,113:92,0,0.143040,0,1,1
6,Atlanta Hawks,Cleveland Cavaliers,31,10,2018,2.71,1.49,114,136,0,0,Cleveland Cavaliers - Atlanta Hawks,136:114,0,0.354762,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2455,Washington Wizards,Denver Nuggets,1,4,2019,5.68,1.15,95,90,0,0,Denver Nuggets - Washington Wizards,90:95,1,0.168375,0,0,1
2456,Washington Wizards,Chicago Bulls,4,4,2019,1.19,5.00,114,115,0,1,Washington Wizards - Chicago Bulls,114:115,0,0.807754,1,0,1
2457,Washington Wizards,San Antonio Spurs,6,4,2019,3.22,1.37,112,129,0,1,Washington Wizards - San Antonio Spurs,112:129,0,0.298475,0,1,1
2458,Washington Wizards,New York Knicks,8,4,2019,1.58,2.44,110,113,0,0,New York Knicks - Washington Wizards,113:110,0,0.606965,1,0,1


In [7]:
# Success rate of all with high prob of winning or loss .

NBA19rest['oddscorrect']= np.where(NBA19rest['winpred'] == NBA19rest['win'] ,1,0)
NBA19rest['oddscorrect'].mean()

0.7450292397660818

As one might expect, the success rate is considerably higher for this subset, which accounts for about 70% of all observations.

## Self Test - 1

Identify the proportion of correct bookmaker predictions when the odds of team win were above 70% or below 30%.

In [10]:
#Your Code Here
NBA19['highprob']= np.where(NBA19['winprob'] >= .7, 1, np.where(NBA19['winprob'] <= .3, 1, 0))
NBA19rest1= NBA19[NBA19['highprob'] ==1 ].copy()
NBA19rest1

,team,opponent,day,month,year,winodds,loseodds,teampts,oppopts,overtime,home,Game(home-away),HAscore,win,winprob,winpred,oddscorrect,highprob
1,Atlanta Hawks,Memphis Grizzlies,20,10,2018,3.34,1.34,117,131,0,0,Memphis Grizzlies - Atlanta Hawks,131:117,0,0.286325,0,1,1
2,Atlanta Hawks,Cleveland Cavaliers,22,10,2018,3.97,1.26,133,111,0,0,Cleveland Cavaliers - Atlanta Hawks,111:133,1,0.240918,0,0,1
5,Atlanta Hawks,Philadelphia 76ers,30,10,2018,6.71,1.12,92,113,0,0,Philadelphia 76ers - Atlanta Hawks,113:92,0,0.143040,0,1,1
9,Atlanta Hawks,Charlotte Hornets,7,11,2018,6.67,1.12,102,113,0,0,Charlotte Hornets - Atlanta Hawks,113:102,0,0.143774,0,1,1
12,Atlanta Hawks,Los Angeles Lakers,12,11,2018,6.38,1.13,106,107,0,0,Los Angeles Lakers - Atlanta Hawks,107:106,0,0.150466,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2450,Washington Wizards,Denver Nuggets,22,3,2019,4.06,1.26,108,113,0,1,Washington Wizards - Denver Nuggets,108:113,0,0.236842,0,1,1
2454,Washington Wizards,Utah Jazz,30,3,2019,8.47,1.08,124,128,0,0,Utah Jazz - Washington Wizards,128:124,0,0.113089,0,1,1
2455,Washington Wizards,Denver Nuggets,1,4,2019,5.68,1.15,95,90,0,0,Denver Nuggets - Washington Wizards,90:95,1,0.168375,0,0,1
2456,Washington Wizards,Chicago Bulls,4,4,2019,1.19,5.00,114,115,0,1,Washington Wizards - Chicago Bulls,114:115,0,0.807754,1,0,1


In [11]:
# Success rate of all with high prob of winning or loss .

NBA19rest['oddscorrect']= np.where(NBA19rest['winpred'] == NBA19rest['win'] ,1,0)
NBA19rest['oddscorrect'].mean()

0.7450292397660818

## Brier Scores

Calculating the success of a forecast, such as the probabilities implied by betting odds, by counting how often the actual outcome was predicted using some threshold probability (e.g. over 50%, over 60%, over 70%) provides a reasonable impression of accuracy, but is obviously sensitive to the threshold selected, while there is no self-evident "correct" threshold. Moreover, each prediction is given a score of one or zero, which is a very crude measure of the closeness of the prediction to the outcome.

The Brier Score, first proposed in the context of weather forecasting by the statistician Glenn Brier in 1950, gives us a precise measure of the degree of accuracy of the predictions. For each possible outcome and forecast, the Brier Score calculates the squared value of the difference, which lies between 0 (100% accurate) and 1 (100% inaccurate). Thus we have a score for each outcome (row) in our data. The Brier Score for the forecasts is the average of the rows.

In the NBA, MLB, and IPL data we have two possible outcomes for each team in each game - win or lose. In the NHL and EPL data we have three possible outcomes for each team in each game - win, lose, or tie/draw.

With two possible outcome the Brier Score for each row is: \begin{align*} (o_1-p_1)^2 + (o_2-p_2)^2\end{align*}

Where o refers to the outcome, outcome 1 is that the team wins ($o_{1}=1$) and outcome 2 is that the team loses ($o_{2}=0$),  and $p_{i}$ (i = 1,2) refers to the forecast probability of each outcome, and $p_{2}$ = 1 - $p_{1}$. 

With three outcomes the Brier Score for each row is: \begin{align*} (o_1-p_1)^2 + (o_2-p_2)^2 + (o_3-p_3)^2\end{align*}

The Brier Score for the forecast model is then just the mean (1/N) of all the rows. 

Lower Brier Scores imply a more accurate forecast.

A Brier Score equal to zero implies 100% accuracy. For the two outcome version, a Brier Score equal to 1 implies perfect inaccuracy, while for the three outcome version a Brier Score of 2 implies perfect inaccuracy.

If each outcome were equally likely, and predictions were random, then the Brier Score in the tow outcome case would be 0.5 ( = $(1 -0.5)^2 + (0 - 0.5)^2$) and in the three outcome case would be 0.666 ( = $(1 -.333)^2 + 2(0 - 0.333)^2$)
<br>
<br>
<br>
[N.B. with two outcomes you will often see the Brier score defined as simply: \begin{align*} (o_1-p_1)^2 \end{align*}

However, given that $p_{2}$ = 1 - $p_{1}$ it is easy to show that: \begin{align*} (o_1-p_1)^2 + (o_2-p_2)^2 = 2(o_1-p_1)^2 \end{align*}

and therefore this is simply a matter of scaling. For the purposes of comparison, we will use the longer version.]




## Brier Score In Two Outcome Leagues

We first look at Brier Scores for the NBA, MLB, and IPL, where each game must result in either a win or a loss for each team. First the NBA:

In [12]:
NBA19['Brier']= (NBA19['win']-NBA19['winprob'])**2 + ((1-NBA19['win'])-(1-NBA19['winprob']))**2
NBA19['Brier'].mean()                                                     

0.40736941528135695

Given that even choosing at random would obtain a Brier Score of 0.5 (the best possible result is zero), this result may not seem particularly impressive. However, recall that when we aggregated the probabilities for each team over the season, the expected win percentage was highly correlated with the actual win percentage. Even if the betting odds for an individual provide only a limited insight into the outcome of an individual game, if these insights are aggregated over a season of 82, they provide a much more reliable picture. This is an example of *the law of large numbers*, which states that the greater the number of trials, the closer the actual result will be to the expected (predicted) outcome.

## Self Test - 2

Generate the Brier Score for the bookmaker probabilities for the 2019 MLB season.

In [13]:
MLB19 = pd.read_excel("../../Data/Week 2/MLB2019odds.xlsx")

MLB19.head()

,team,opponent,day,month,home,oddsfor,oddsagainst,runsfor,runsagainst,win,game home team- away team,score,date
0,Arizona Diamondbacks,Atlanta Braves,10,5,1,1.91,1.92,3,2,1,Arizona Diamondbacks - Atlanta Braves,03:02:00,10.05.
1,Arizona Diamondbacks,Atlanta Braves,11,5,1,1.73,2.14,1,2,0,Arizona Diamondbacks - Atlanta Braves,01:02:00,11.05.
2,Arizona Diamondbacks,Atlanta Braves,12,5,1,2.03,1.82,3,5,0,Arizona Diamondbacks - Atlanta Braves,03:05:00,12.05.
3,Arizona Diamondbacks,Atlanta Braves,12,5,1,2.06,1.79,4,6,0,Arizona Diamondbacks - Atlanta Braves,04:06:00,12.05.
4,Arizona Diamondbacks,Baltimore Orioles,23,7,1,1.39,3.08,6,3,1,Arizona Diamondbacks - Baltimore Orioles,06:03:00,23.07.


In [14]:
MLB19['winprob']= 1/ MLB19.oddsfor / (1/ MLB19.oddsfor + 1/MLB19.oddsagainst)
# (win - winprob **2) + ((1- win) - (1-win_prob))**2
MLB19['Brier']= (MLB19['win'] - MLB19['winprob']) ** 2 + ((1- MLB19['win'])- (1-MLB19['winprob']))*2
MLB19.Brier.mean()

0.23569105314523076

Let's now calculate the Brier Score for the Indian Premier League. Given the poor fit between expected wins and actual wins that we found before, we might expect that the Brier Score will be low too. It is worse than the Brier Score for the other two leagues, and in fact is slightly larger than the expected Brier score if predictions were random. Unlike the other two leagues, the bookmakers seems totally unable to predict outcomes in the IPL:

In [20]:
IPL18 = pd.read_excel("../../Data/Week 2/IPL2018odds.xlsx")
IPL18.head()

,date,team,oppo,home,teamwinodds,oppowinodds,teamscore,opposcore,teamwin,playoff,DLS result,home-away teams,score
0,2018-04-07 00:00:00,Chennai Super Kings,Mumbai Indians,0,104,-130,169,165,1,0,0,Mumbai Indians - Chennai Super Kings,165:169
1,2018-04-08 00:00:00,Kings XI Punjab,Delhi Capitals,0,-143,114,167,166,1,0,0,Delhi Capitals - Kings XI Punjab,166:167
2,2018-04-08 00:00:00,Royal Challengers Bangalore,Kolkata Knight Riders,0,-143,114,176,177,0,0,0,Kolkata Knight Riders - Royal Challengers Bang...,177:176
3,2018-04-09 00:00:00,Rajasthan Royals,Sunrisers Hyderabad,0,144,-179,125,127,0,0,0,Sunrisers Hyderabad - Rajasthan Royals,127:125
4,2018-04-10 00:00:00,Kolkata Knight Riders,Chennai Super Kings,0,-104,-118,202,205,0,0,0,Chennai Super Kings - Kolkata Knight Riders,205:202


In [15]:
IPL18 = pd.read_excel("../../Data/Week 2/IPL2018odds.xlsx")
IPL18['winprobX']= np.where(IPL18['teamwinodds']>0, 100/(100+IPL18['teamwinodds']),\
                           -IPL18['teamwinodds']/(100-IPL18['teamwinodds']))
IPL18['loseprobX']= np.where(IPL18['oppowinodds']>0, 100/(100+IPL18['oppowinodds']),\
                           -IPL18['oppowinodds']/(100-IPL18['oppowinodds']))
IPL18['winprob']=IPL18['winprobX']/(IPL18['winprobX']+IPL18['loseprobX'])
IPL18['Brier']= (IPL18['teamwin']-IPL18['winprob'])**2 + ((1-IPL18['teamwin'])-(1-IPL18['winprob']))**2
IPL18['Brier'].mean()


0.5045397630021412

## Brier Scores in Three Outcome Leagues

In the NHL teams are awared two points for a win, one for an overtime loss, and zero points for a loss in regular time. In the EPL teams are awarded three points for a win, one point for a draw and zero points for a loss. We looked already at the relationship between points won and expected points (based on the bookmaker probabilities) in these leagues, so now we look at the Brier Scores in these three outcome leagues.

NHL:

In [12]:
NHL19 = pd.read_excel("../../Data/Week 2/NHL2018-19odds.xlsx")
NHL19

,team,opponent,day,month,year,home,winodds,tieodds,loseodds,teamgoals,oppogoals,overtime,pen,win,loss,OTL,game-ht-at,scoreht-at
0,San Jose Sharks,Anaheim Ducks,4,10,2018,1,1.87,4.08,3.50,2,5,0,0,0,1,0,San Jose Sharks - Anaheim Ducks,02:05:00
1,Toronto Maple Leafs,Montreal Canadiens,4,10,2018,1,1.65,4.57,4.21,3,2,1,0,1,0,0,Toronto Maple Leafs - Montreal Canadiens,3:2 ET
2,Vancouver Canucks,Calgary Flames,4,10,2018,1,2.82,4.03,2.17,5,2,0,0,1,0,0,Vancouver Canucks - Calgary Flames,05:02:00
3,Washington Capitals,Boston Bruins,4,10,2018,1,2.32,3.97,2.62,7,0,0,0,1,0,0,Washington Capitals - Boston Bruins,07:00:00
4,Buffalo Sabres,Boston Bruins,5,10,2018,1,2.79,3.98,2.19,0,4,0,0,0,1,0,Buffalo Sabres - Boston Bruins,00:04:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2537,Columbus Blue Jackets,Ottawa Senators,7,4,2019,0,1.82,4.29,3.55,6,2,0,0,1,0,0,Ottawa Senators - Columbus Blue Jackets,02:06:00
2538,Carolina Hurricanes,Philadelphia Flyers,7,4,2019,0,2.06,4.26,2.88,4,3,0,0,1,0,0,Philadelphia Flyers - Carolina Hurricanes,03:04:00
2539,New York Rangers,Pittsburgh Penguins,7,4,2019,0,4.95,4.95,1.51,4,3,1,0,1,0,0,Pittsburgh Penguins - New York Rangers,3:4 ET
2540,Colorado Avalanche,San Jose Sharks,7,4,2019,0,3.21,4.21,1.94,2,5,0,0,0,1,0,San Jose Sharks - Colorado Avalanche,05:02:00


In [13]:
NHL19['winprob']= 1/(NHL19['winodds'])/(1/(NHL19['winodds'])+ 1/(NHL19['tieodds'])+ 1/(NHL19['loseodds']))
NHL19['tieprob']= 1/(NHL19['tieodds'])/(1/(NHL19['winodds'])+ 1/(NHL19['tieodds'])+ 1/(NHL19['loseodds']))
NHL19

,team,opponent,day,month,year,home,winodds,tieodds,loseodds,teamgoals,oppogoals,overtime,pen,win,loss,OTL,game-ht-at,scoreht-at,winprob,tieprob
0,San Jose Sharks,Anaheim Ducks,4,10,2018,1,1.87,4.08,3.50,2,5,0,0,0,1,0,San Jose Sharks - Anaheim Ducks,02:05:00,0.501852,0.230016
1,Toronto Maple Leafs,Montreal Canadiens,4,10,2018,1,1.65,4.57,4.21,3,2,1,0,1,0,0,Toronto Maple Leafs - Montreal Canadiens,3:2 ET,0.570459,0.205964
2,Vancouver Canucks,Calgary Flames,4,10,2018,1,2.82,4.03,2.17,5,2,0,0,1,0,0,Vancouver Canucks - Calgary Flames,05:02:00,0.333412,0.233306
3,Washington Capitals,Boston Bruins,4,10,2018,1,2.32,3.97,2.62,7,0,0,0,1,0,0,Washington Capitals - Boston Bruins,07:00:00,0.404878,0.236604
4,Buffalo Sabres,Boston Bruins,5,10,2018,1,2.79,3.98,2.19,0,4,0,0,0,1,0,Buffalo Sabres - Boston Bruins,00:04:00,0.336137,0.235634
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2537,Columbus Blue Jackets,Ottawa Senators,7,4,2019,0,1.82,4.29,3.55,6,2,0,0,1,0,0,Ottawa Senators - Columbus Blue Jackets,02:06:00,0.516284,0.219030
2538,Carolina Hurricanes,Philadelphia Flyers,7,4,2019,0,2.06,4.26,2.88,4,3,0,0,1,0,0,Philadelphia Flyers - Carolina Hurricanes,03:04:00,0.454784,0.219919
2539,New York Rangers,Pittsburgh Penguins,7,4,2019,0,4.95,4.95,1.51,4,3,1,0,1,0,0,Pittsburgh Penguins - New York Rangers,3:4 ET,0.189460,0.189460
2540,Colorado Avalanche,San Jose Sharks,7,4,2019,0,3.21,4.21,1.94,2,5,0,0,0,1,0,San Jose Sharks - Colorado Avalanche,05:02:00,0.292645,0.223133


In [14]:
NHL19['Brier']= (NHL19['win']-NHL19['winprob'])**2 + ((NHL19['OTL'])-(NHL19['tieprob']))**2 \
+((NHL19['loss'])-(1 -NHL19['winprob']- NHL19['tieprob']))**2
NHL19['Brier'].mean() 

0.5898410611896546

In the three outcome case, the Brier Score if each probability is chosen randomly equals 0.666, while the perfect forecast (as in the two outcome case) is zero. Thus, as with the NBA and MLB it might appear that the Brier Score is not especially good, also recall that the correlation between actual points and expected points for each team across the season was very high. 

## Self Test - 3

Now calculate the Brier score for the EPL.

In [34]:
EPL19 = pd.read_excel("../../Data/Week 2/EPL2018-19odds.xlsx")

EPL19.head()

,Round,day,month,year,home,team,oppo,home.1,winodds,drawodds,loseodds,teamgoals,oppogoals,pts,matchup (ht first)
0,1. ROUND,10,8,2018,1,Manchester Utd,Leicester,1,1.55,3.94,7.14,2,1,3,Manchester Utd - Leicester
1,1. ROUND,10,8,2018,0,Leicester,Manchester Utd,0,7.14,3.94,1.55,1,2,0,Manchester Utd - Leicester
2,1. ROUND,11,8,2018,1,Bournemouth,Cardiff,1,1.86,3.59,4.48,2,0,3,Bournemouth - Cardiff
3,1. ROUND,11,8,2018,1,Watford,Brighton,1,2.48,3.09,3.19,2,0,3,Watford - Brighton
4,1. ROUND,11,8,2018,1,Fulham,Crystal Palace,1,2.55,3.38,2.84,0,2,0,Fulham - Crystal Palace


In [35]:
EPL19['win']= np.where(EPL19['pts'] == 3,1,0)
EPL19['draw']= np.where(EPL19.pts == 1, 1, 0)
EPL19['loss']= np.where(EPL19.pts == 0, 1, 0)
EPL19.head()

,Round,day,month,year,home,team,oppo,home.1,winodds,drawodds,loseodds,teamgoals,oppogoals,pts,matchup (ht first),win,draw,loss
0,1. ROUND,10,8,2018,1,Manchester Utd,Leicester,1,1.55,3.94,7.14,2,1,3,Manchester Utd - Leicester,1,0,0
1,1. ROUND,10,8,2018,0,Leicester,Manchester Utd,0,7.14,3.94,1.55,1,2,0,Manchester Utd - Leicester,0,0,1
2,1. ROUND,11,8,2018,1,Bournemouth,Cardiff,1,1.86,3.59,4.48,2,0,3,Bournemouth - Cardiff,1,0,0
3,1. ROUND,11,8,2018,1,Watford,Brighton,1,2.48,3.09,3.19,2,0,3,Watford - Brighton,1,0,0
4,1. ROUND,11,8,2018,1,Fulham,Crystal Palace,1,2.55,3.38,2.84,0,2,0,Fulham - Crystal Palace,0,0,1


In [36]:
EPL19['winprob']= 1/(EPL19['winodds'])/( 1/(EPL19['winodds'])+  1/(EPL19['drawodds'])+ 1/(EPL19['loseodds']) )
EPL19['drawprob']= 1/(EPL19['drawodds'])/( 1/(EPL19['winodds'])+ 1/(EPL19['drawodds'])+ 1/(EPL19['loseodds']) )

In [37]:
EPL19['Brier']= (EPL19['win']-EPL19['winprob'])**2 + ((EPL19['draw'])-(EPL19['drawprob']))**2 \
+((EPL19['loss'])-(1 -EPL19['winprob']- EPL19['drawprob']))**2
EPL19['Brier'].mean() 

0.5212400786602575

## Conclusion

In this notebook we have applied the Brier Score to the evaluation of bookmaker forecasts. In the next week we will use these Brier Scores as a benchmark for the evaluation of our own forecasts.

In this week we have looked at betting markets in relation to sports, considered the way in which the market for betting works, and explained the different measures of betting odds, and some of the popular ways in which people bet on sports. We then introduced some odds data for different leagues and examined ways of measuring the accuracy of odds. 

Bookmaker odds are not always very accurate when it comes individual games, but they seem to be fairly reliable in most sports when aggregated over a large number of games. The exception in our data was the Indian Premier League, where the odds seem to provide very little insight into outcomes. It should be recalled from the first course in this series that we also found that wage data was a very reliable indicator of success in all of our leagues apart from the Indian Premier League. This is worth bearing in mind when moving on to prediction in the next week of this course.